# Solidity语法

声明语法版本
```solidity

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.13;
```

## 基础语法

### if else



### for,while



### Mapping

- mapping(address => uint) public myMap;
  1. keyType：string、address等
  2. valueType：anyType
  3. 空则返回默认值

#### 示例
```solidity
contract Mapping {
    // Mapping from address to uint
    mapping(address => uint) public myMap;

    function get(address _addr) public view returns (uint) {
        return myMap[_addr];
    }

    function set(address _addr, uint _i) public {
        myMap[_addr] = _i;
    }
    function remove(address _addr) public {
        delete myMap[_addr];
    }
}
```

### Array

#### 示例
```solidity

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.13;

contract Array {
    // Several ways to initialize an array
    uint[] public arr;
    uint[] public arr2 = [1, 2, 3];
    uint[10] public myFixedSizeArr; //固定size，default value = 0

    function get(uint i) public view returns (uint) {
        return arr[i];
    }

    // not recommend
    function getArr() public view returns (uint[] memory) {
        return arr;
    }
    // append ele
    function push(uint i) public {
        arr.push(i);
    }
    // pop ele
    function pop() public {
        arr.pop();
    }
    function getLength() public view returns (uint) {
        return arr.length;
    }
    // delete means set 0
    function remove(uint index) public {
        delete arr[index];
    }
    function examples() external {
        // 内存创建，仅能固定大小
        uint[] memory a = new uint[](5);
    }
}
```



### Enum
方便模式选择和状态跟随，支持contract导入。类似`proto`

#### 示例
```solidity
contract Enum {
    // 状态转移
    enum Status {
        Pending, //0
        Shipped, //1
        Accepted,//2
        Rejected,//3
        Canceled //4
    }
    // 默认声明第一个status
    Status public status;
    function get() public view returns (Status) {
        return status;
    }

    // 传入uint更新状态
    function set(Status _status) public {
        status = _status;
    }

    // func对应status转换
    function cancel() public {
        status = Status.Canceled;
    }
    // 状态置0
    function reset() public {
        delete status;
    }
}

```

### Structs
创建结构体自定义类型，整体组织相关数据，类似Enum定义支持contract导入

#### 示例
```solidity
contract Todos {
    struct Todo {
        string text;
        bool completed; //default false
    }
    Todo[] public todos; // 声明struct array

    function create(string calldata _text) public {
        todos.push(Todo(_text, false)); //1. func easy create, calling it like a function
        todos.push(Todo({text: _text, completed: false})); //2. map create, key value mapping
        Todo memory todo;  // 3. initialize an empty struct and then update it
        todo.text = _text;
        todos.push(todo);
    }

    function get(uint _index) public view returns (string memory text, bool completed) {
        Todo storage todo = todos[_index];
        return (todo.text, todo.completed);
    }

    // update text for todos[_index]
    function updateText(uint _index, string calldata _text) public {
        Todo storage todo = todos[_index];
        todo.text = _text;
    }

    // update completed
    function toggleCompleted(uint _index) public {
        Todo storage todo = todos[_index];
        todo.completed = !todo.completed;
    }
}
```

### Data Locations
Variables声明storage、memory、calldata指定数据地址。

- storage：state variable (状态变量存储于blockchain)
- memory：内存中函数生命周期调用
- calldata：指定函数参数存储

#### 示例
```solidity
contract DataLocations {
    uint[] public arr;
    mapping(uint => address) map;
    struct MyStruct {
        uint foo;
    }
    mapping(uint => MyStruct) myStructs;

    function f() public {
        // 函数调用state variables。在contract内声明，默认storage state variable
        _f(arr, map, myStructs[1])；
        MyStruct storage myStruct = myStructs[1];
        MyStruct memory myMemStruct = MyStruct(0); // create a struct in memory
    }
    function _f(
        uint[] storage _arr,
        mapping(uint => address) storage _map,
        MyStruct storage _myStruct
    ) internal {
        // do something with storage variables
    }
    function g(uint[] memory _arr) public returns (uint[] memory) { // return memory variables
        // do something with memory array
    }
    function h(uint[] calldata _arr) external {
        // do something with calldata array
    }
}
```

### Function


#### 示例
```solidity
contract Function {
    // Functions can return multiple values.
    function returnMany() public pure returns (uint x, bool b, uint y){ //可不需指定return name
        return (1, true, 2);

        //可对var name赋值作为return
        //x = 1;
        //b = true;
        //y = 2;
    }
    (uint i, bool b, uint j) = returnMany(); //指定dataType

    // 不可传入传出map

    // 可传入array
    function arrayInput(uint[] memory _arr) public {}

    // return state variables
    uint[] public arr;
    function arrayOutput() public view returns (uint[] memory) {
        return arr;
    }
}
```

### View and Pure Functions

- `View function` 不改变状态，
- `Pure function` 不改变状态，也不读

#### 示例
```solidity
contract ViewAndPure {
    uint public x = 1;
    function addToX(uint y) public view returns (uint) { // 不改变，状态
        return x + y;
    }
    function add(uint i, uint j) public pure returns (uint) { // 不改变，不读 状态
        return i + j;
    }
}
```

### Error
交易中error，撤销已改状态。`error`类型包括：**require, revert or assert**

- `require` 执行前验证
- `revert` 类似require
- `assert` 代码检查。控制流程节省gas

#### 示例
```solidity
contract Error {
    function testRequire(uint _i) public pure {
        require(_i > 10, "Input must be greater than 10"); // Require  用于检查 inputs、执行前条件、其他函数返回等
    }
    function testRevert(uint _i) public pure {
        if (_i <= 10) {
            revert("Input must be greater than 10"); //复杂使用
        }
    }
    uint public num;
    function testAssert() public view {
        assert(num == 0); // Assert 用于检查internal errors。view声明function不改变状态，故检查num值的改动
    }
    // 自定义声明error
    error InsufficientBalance(uint balance, uint withdrawAmount);
    function testCustomError(uint _withdrawAmount) public view {
        uint bal = address(this).balance;
        if (bal < _withdrawAmount) {
            revert InsufficientBalance({balance: bal, withdrawAmount: _withdrawAmount});
        }
    }
}
```

## 数据类型

### uint

- uint8   ranges from 0 to 2 ** 8 - 1
- uint16  ranges from 0 to 2 ** 16 - 1
- ...
- uint256 ranges from 0 to 2 ** 256 - 1, default

### int
- int256 ranges from -2 ** 255 to 2 ** 255 - 1, default
- int128 ranges from -2 ** 127 to 2 ** 127 - 1
- int public minInt = type(int).min;
- int public maxInt = type(int).max;

### address
- public addr = 0xCA35b7d915458EF540aDe6068dFe2F44E8fa733c;


### byte(byte[])

- bytes1 a = 0xb5; //  [10110101]

### Default values
- bool public defaultBoo; // false
- uint public defaultUint; // 0
- int public defaultInt; // 0
- address public defaultAddr; // 0x0000000000000000000000000000000000000000


## 变量

### local（本地变量）
- 函数内声明
- 不存储于链中

### state（状态变量）
- 函数外声明
- 存储于链中

### global（全局变量）
- 链中上下文信息

#### 示例
```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.13;

contract Variables {
    // State variables are stored on the blockchain.
    string public text = "Hello";
    uint public num = 123;

    function doSomething() public {
        // Local variables are not saved to the blockchain.
        uint i = 456;

        // Here are some global variables
        uint timestamp = block.timestamp; // Current block timestamp
        address sender = msg.sender; // address of the caller
    }
}

```

### constant（常量）
大写声明常量类型，节省gas cost

- address public constant MY_ADDRESS = 0x777788889999AaAAbBbbCcccddDdeeeEfFFfCcCc;
- uint public constant MY_UINT = 123;

### Immutable（不可变量）
和constant相似，但是允许构造函数中修改。示例：

```solidity
contract Immutable {
    address public immutable MY_ADDRESS;
    uint public immutable MY_UINT;
    constructor(uint _myUint) {
        MY_ADDRESS = msg.sender;
        MY_UINT = _myUint;
    }
}
```

### 变量-读与写
- 变量的写和更新，需要发送交易
- 变量的读，不需要发送交易和交易费用

## Ether coin相关


### ether and wei
单位 1 ehter == 1e18 wei

- uint public oneWei = 1 wei; //1 wei is equal to 1
- uint public oneEther = 1 ether; // 1 ether is equal to 10^18 wei


### Gas

#### Gas计算
**gas = gas spent * gas price  ether**

- `gas spent` 交易中花费的gas总量
- `gas price` gas单价

区块中，`gas price`越高有更高的优先权，未用完退回。

#### Gas Limit
花费gas的上下界

- `gas limit` 交易时最大gas量，自定义设定
- `block gas limit` 块中允许的最大gas量, 私链设定

## 二、

### function modifier

